In [ ]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import pandas_datareader.data as web
import requests
import datetime
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In order to do a discounted cash flow (DCF) analysis we need to fetch data from actual income statements and balance sheet from the company to get an estimated statement of free cash flows (FCF).  Because Disney had a negative cash flow and negative income in 2020, we chose to do our analysis using 2019 numbers and using 2020 as the first year of forecasted values.  Below you will see an "A" after the year for actual values and an "F" for forecasted.  Income statement was pulled using the Fianancial Modeling Prep API. The Average growth over the last three years was calculated using the last three years of income statments.  Forecast values were predicted using the income statement from 2019 as a percent of revenue (as_%_of_revenue).

In [ ]:
api_key = os.getenv("FMP")
api_keyalpha = os.getenv("alpha_vantage")
company = 'DIS'

In [ ]:
#Five year projection based on 2019 being the initial year

IS= requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={api_key}').json()

#Calculation of average revenue growth for the last 3 years.
revenue_growth = (IS[0]['revenue'] - IS[1]['revenue']) / IS[1]['revenue']
revenue_growth1 = (IS[1]['revenue'] - IS[2]['revenue']) / IS[2]['revenue']
revenue_growth2 = (IS[2]['revenue'] - IS[3]['revenue']) / IS[3]['revenue']
#avg_growth = (revenue_growth + revenue_growth1 + revenue_growth2) / 3
#We calculated an average growth rate, but decided to use -0.02 and 0.05 as min and max values so these will have to be manually input below:

avg_growth = -0.02


#Forecast Income statement from 2020 - 2024.
net_income = IS[1]['netIncome']
income_statement = pd.DataFrame.from_dict(IS[1],orient='index')
income_statement = income_statement[6:28]
income_statement.columns = ['2019A']
income_statement['as_%_of_revenue'] = income_statement / income_statement.iloc[0]

income_statement['2020F'] =  (income_statement['2019A']['revenue'] * (1+avg_growth)) * income_statement['as_%_of_revenue'] 
income_statement['2021F'] =  (income_statement['2020F']['revenue'] * (1+avg_growth)) * income_statement['as_%_of_revenue'] 
income_statement['2022F'] =  (income_statement['2021F']['revenue'] * (1+avg_growth)) * income_statement['as_%_of_revenue'] 
income_statement['2023F'] =  (income_statement['2022F']['revenue'] * (1+avg_growth)) * income_statement['as_%_of_revenue'] 
income_statement['2024F'] =  (income_statement['2023F']['revenue'] * (1+avg_growth)) * income_statement['as_%_of_revenue'] 
income_statement

In [ ]:
#Fetch Balance sheet data from 2019.
BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={api_key}').json()

balance_sheet = pd.DataFrame.from_dict(BS[1],orient='index')
balance_sheet = balance_sheet[6:-2]
balance_sheet.columns = ['2019A']
balance_sheet['as_%_of_revenue'] = balance_sheet / income_statement['2019A'].iloc[0]

#Forecasting Balance Sheet from 2020 - 2024.
balance_sheet['2020F'] =  income_statement['2020F']['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['2021F'] =  income_statement['2021F']['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet['2022F'] =  income_statement['2022F']['revenue'] * balance_sheet['as_%_of_revenue'] 
balance_sheet['2023F'] =  income_statement['2023F']['revenue'] * balance_sheet['as_%_of_revenue'] 
balance_sheet['2024F'] =  income_statement['2024F']['revenue'] * balance_sheet['as_%_of_revenue']
balance_sheet

In [ ]:
#Statment of Cash Flows for 2020 - 2024 predicted using items from the income statement and balance sheet
CF_forecast = {}
CF_forecast['2020F'] = {}
CF_forecast['2020F']['netIncome'] = income_statement['2020F']['netIncome']
CF_forecast['2020F']['inc_depreciation'] = income_statement['2020F']['depreciationAndAmortization']
CF_forecast['2020F']['inc_receivables'] = balance_sheet['2020F']['netReceivables'] - balance_sheet['2019A']['netReceivables']
CF_forecast['2020F']['inc_inventory'] = balance_sheet['2020F']['inventory'] - balance_sheet['2019A']['inventory']
CF_forecast['2020F']['inc_payables'] = balance_sheet['2020F']['accountPayables'] - balance_sheet['2019A']['accountPayables']
CF_forecast['2020F']['CF_operations'] = CF_forecast['2020F']['netIncome'] + CF_forecast['2020F']['inc_depreciation'] + (CF_forecast['2020F']['inc_receivables'] * -1) + (CF_forecast['2020F']['inc_inventory'] *-1) + CF_forecast['2020F']['inc_payables']
CF_forecast['2020F']['CAPEX'] = balance_sheet['2020F']['propertyPlantEquipmentNet'] - balance_sheet['2019A']['propertyPlantEquipmentNet'] + income_statement['2020F']['depreciationAndAmortization']
CF_forecast['2020F']['FCF'] = CF_forecast['2020F']['CAPEX'] + CF_forecast['2020F']['CF_operations']


CF_forecast['2021F'] = {}
CF_forecast['2021F']['netIncome'] = income_statement['2021F']['netIncome']
CF_forecast['2021F']['inc_depreciation'] = income_statement['2021F']['depreciationAndAmortization']
CF_forecast['2021F']['inc_receivables'] = balance_sheet['2021F']['netReceivables'] - balance_sheet['2020F']['netReceivables']
CF_forecast['2021F']['inc_inventory'] = balance_sheet['2021F']['inventory'] - balance_sheet['2020F']['inventory']
CF_forecast['2021F']['inc_payables'] = balance_sheet['2021F']['accountPayables'] - balance_sheet['2020F']['accountPayables']
CF_forecast['2021F']['CF_operations'] = CF_forecast['2021F']['netIncome'] + CF_forecast['2021F']['inc_depreciation'] + (CF_forecast['2021F']['inc_receivables'] * -1) + (CF_forecast['2021F']['inc_inventory'] *-1) + CF_forecast['2021F']['inc_payables']
CF_forecast['2021F']['CAPEX'] = balance_sheet['2021F']['propertyPlantEquipmentNet'] - balance_sheet['2020F']['propertyPlantEquipmentNet'] + income_statement['2021F']['depreciationAndAmortization']
CF_forecast['2021F']['FCF'] = CF_forecast['2021F']['CAPEX'] + CF_forecast['2021F']['CF_operations']


CF_forecast['2022F'] = {}
CF_forecast['2022F']['netIncome'] = income_statement['2022F']['netIncome']
CF_forecast['2022F']['inc_depreciation'] = income_statement['2022F']['depreciationAndAmortization'] 
CF_forecast['2022F']['inc_receivables'] = balance_sheet['2022F']['netReceivables'] - balance_sheet['2021F']['netReceivables']
CF_forecast['2022F']['inc_inventory'] = balance_sheet['2022F']['inventory'] - balance_sheet['2021F']['inventory']
CF_forecast['2022F']['inc_payables'] = balance_sheet['2022F']['accountPayables'] - balance_sheet['2021F']['accountPayables']
CF_forecast['2022F']['CF_operations'] = CF_forecast['2022F']['netIncome'] + CF_forecast['2022F']['inc_depreciation'] + (CF_forecast['2022F']['inc_receivables'] * -1) + (CF_forecast['2022F']['inc_inventory'] *-1) + CF_forecast['2022F']['inc_payables']
CF_forecast['2022F']['CAPEX'] = balance_sheet['2022F']['propertyPlantEquipmentNet'] - balance_sheet['2021F']['propertyPlantEquipmentNet'] + income_statement['2022F']['depreciationAndAmortization']
CF_forecast['2022F']['FCF'] = CF_forecast['2022F']['CAPEX'] + CF_forecast['2022F']['CF_operations']


CF_forecast['2023F'] = {}
CF_forecast['2023F']['netIncome'] = income_statement['2023F']['netIncome']
CF_forecast['2023F']['inc_depreciation'] = income_statement['2023F']['depreciationAndAmortization']
CF_forecast['2023F']['inc_receivables'] = balance_sheet['2023F']['netReceivables'] - balance_sheet['2022F']['netReceivables']
CF_forecast['2023F']['inc_inventory'] = balance_sheet['2023F']['inventory'] - balance_sheet['2022F']['inventory']
CF_forecast['2023F']['inc_payables'] = balance_sheet['2023F']['accountPayables'] - balance_sheet['2022F']['accountPayables']
CF_forecast['2023F']['CF_operations'] = CF_forecast['2023F']['netIncome'] + CF_forecast['2023F']['inc_depreciation'] + (CF_forecast['2023F']['inc_receivables'] * -1) + (CF_forecast['2023F']['inc_inventory'] *-1) + CF_forecast['2023F']['inc_payables']
CF_forecast['2023F']['CAPEX'] = balance_sheet['2023F']['propertyPlantEquipmentNet'] - balance_sheet['2022F']['propertyPlantEquipmentNet'] + income_statement['2023F']['depreciationAndAmortization']
CF_forecast['2023F']['FCF'] = CF_forecast['2023F']['CAPEX'] + CF_forecast['2023F']['CF_operations']

CF_forecast['2024F'] = {}
CF_forecast['2024F']['netIncome'] = income_statement['2024F']['netIncome']
CF_forecast['2024F']['inc_depreciation'] = income_statement['2024F']['depreciationAndAmortization'] 
CF_forecast['2024F']['inc_receivables'] = balance_sheet['2024F']['netReceivables'] - balance_sheet['2023F']['netReceivables']
CF_forecast['2024F']['inc_inventory'] = balance_sheet['2024F']['inventory'] - balance_sheet['2023F']['inventory']
CF_forecast['2024F']['inc_payables'] = balance_sheet['2024F']['accountPayables'] - balance_sheet['2023F']['accountPayables']
CF_forecast['2024F']['CF_operations'] = CF_forecast['2024F']['netIncome'] + CF_forecast['2024F']['inc_depreciation'] + (CF_forecast['2024F']['inc_receivables'] * -1) + (CF_forecast['2024F']['inc_inventory'] *-1) + CF_forecast['2024F']['inc_payables']
CF_forecast['2024F']['CAPEX'] = balance_sheet['2024F']['propertyPlantEquipmentNet'] - balance_sheet['2023F']['propertyPlantEquipmentNet'] + income_statement['2024F']['depreciationAndAmortization']
CF_forecast['2024F']['FCF'] = CF_forecast['2024F']['CAPEX'] + CF_forecast['2024F']['CF_operations']

CF_fiveyear = pd.DataFrame.from_dict(CF_forecast,orient='columns')
print(CF_fiveyear)

In order to do the DCF analysis the weighted average cost of capital (wacc) needs to be calcualted (Formula below).  WACC can be estimated or calcuate it in several ways, we have chosen to calculate it it using the equation below.

WACC = Kd * (1 -Tc) * (D /D+E) + Ke * (E /D+E)
Where Kd = the cost of debt
Tc = tax rate (what is referred to below as effect tax rate, ETR)
Ke = Cost of equity
D/D+E = Proportion of debt in firm capital structure (items retrieved from the balance sheet)
E/D+E = Proporation of equity in firm capital structure

The first term, Kd, is calculated by Kd = Rf + Credit Spread
rf = the risk free rate
credit spread = is determined by the firm's credit rating which can be calculated using the interest coverage ratio.

In order to get this value the interest coverage and risk free rate need to be calcualted.  The interest coverage ratio is simply EBIT divided by the interest expense (items retrieved from the income statement).  For the risk free rate (RF), which is defined as the theoretical rate of return of an investment with zero risk, we can estimate using the rate of return for one year t-bills which is retrieved using pandas data reader.  This will be used to estimate the cost of debt.

In [ ]:
#Function to caluculate interest coverage ratio from items on most recent income statement
def interest_coverage_and_RF(company):
    IS= requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={api_key}').json()
    EBIT= IS[1]['ebitda'] - IS[1]['depreciationAndAmortization'] 
    interest_expense = IS[1]['interestExpense']
    interest_coverage_ratio = EBIT / interest_expense
    
#Calculation of risk free rate using most recent data for one year treasury bills
    start = datetime.datetime(2019, 7, 12)
    end = datetime.datetime(2020, 7, 12)
    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF/100
    print(RF, interest_coverage_ratio)
    return [RF,interest_coverage_ratio]

interest = interest_coverage_and_RF(company)
RF = interest[0]
interest_coverage_ratio = interest[1]

The following takes in the interest coverage ratio calculated above to determine the credit rating of the company and thus determine their credit spread.  This table can be found at http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ratings.htm. Having determined the credit spread we can calculate Kd (rf + credit spread)

In [ ]:
#Rating is AAA
credit_spread = 0.0063
cost_of_debt = RF + credit_spread
ke = cost_of_debt

Cost of equity (ke) which investopedia defines as the return a company requires to decide if an investment meets capital return requirements.  This is calculated using the company beta which is pulled from the api, the rf rate calculated above, and using the 2019 - 2020 return on the S&P500 which is again pulled from the pandas data reader library.

In [ ]:
beta = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{company}?apikey={api_key}').json()
beta = float(beta['profile']['beta'])
    
#Market Return
start = datetime.datetime(2019, 7, 10)
end = datetime.datetime(2020, 7, 10)
    
SP500 = web.DataReader(['sp500'], 'fred', start, end)
SP500.dropna(inplace = True)
    
SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[-252])-1
    
cost_of_equity = RF+(beta*(SP500yearlyreturn - RF))
print(cost_of_equity)

ke = cost_of_equity

Finally, the effective tax rate (ETR) is pull from the financial ratios (FR) portion of the api.

In [ ]:
FR = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={api_key}').json()
ETR = FR[1]['effectiveTaxRate']
    
    #Capital structure calculated from items on balance sheet (BS)
BS = requests.get(f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={company}&apikey={api_keyalpha}').json()
Debt_to = int(BS['annualReports'][1]['shortLongTermDebtTotal']) / (int(BS['annualReports'][1]['shortLongTermDebtTotal']) + int(BS['annualReports'][1]['totalShareholderEquity']))
equity_to = int(BS['annualReports'][1]['totalShareholderEquity']) / (int(BS['annualReports'][1]['shortLongTermDebtTotal']) + int(BS['annualReports'][1]['totalShareholderEquity']))
wacc = (kd*(1-ETR)*Debt_to) + (ke*equity_to)

Having the wacc in hand we can use the predicted cashflows and calculate the net present value (npv) using numpy financial.

In [ ]:
FCF_list = CF_fiveyear.iloc[-1].values.tolist()
npv = npf.npv(wacc, FCF_list)
npv

Obviously, a company will probably not stop growing after the time we have forecasted above so a terminal value is calculated assuming longterm growth (perpetuity model). We assumed a 2% long-term growth which seems to be standard, but can change.  This terminal value was then dicounted to the present using the wacc calculated as the discount rate.

In [ ]:
#Terminal Value

lt_growth = 0.02

terminal_value = (CF_forecast['2024F']['FCF'] * (1+ lt_growth)) /(wacc  - lt_growth)
discounted_tv = terminal_value/(1+wacc)**4
discounted_tv

In [ ]:
target_equity_value = discounted_tv + npv
debt = balance_sheet['2019A']['totalDebt']
target_value = target_equity_value - debt

num_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={api_key}').json()
num_shares = num_shares[0]['numberOfShares']

target_price_per_share = target_value/num_shares
target_price_per_share

The code above can be run for the three evaluated growth percentages (pessimistic, neutral, optimistic: -2%, 2%, and 5%, repsectively) to obtain a range of predicted stock prices.

In [ ]:
df_data = {'Valuation Method':['DCF\n($321.27 - 505.17)', 'EV/EBITDA\n($84.45 - 128.69)','Monte Carlo Sim\n($63.42 - 916.11)', '5yr-Range\n($85.76 - 201.91)'],
          'Upper': [505.17, 128.69, 63.42, 201.91],
          'Lower': [321.27, 84.45, 916.11, 85.76]}
df = pd.DataFrame(df_data)
df['Height'] = df['Upper'] - df['Lower']

ax = df.plot(kind='bar', y='Height', x='Valuation Method', bottom=df['Lower'], color='red', legend=False)
ax.axhline(y=181.60, xmin=0, xmax=1, marker='_', color='black')
ax.set_ylim(ymin=0, ymax=1100)
ax.set_ylabel('Stock Price/Share')
plt.tight_layout()
plt.show()

Code for Bollinger Bands

In [ ]:
dis_closing_prices = pd.read_csv('DIS.csv')
dis_closing_prices.set_index('Date', inplace=True)
dis_closing_prices.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
dis_closing_prices.describe()

dis_closing_prices['mavg30'] = dis_closing_prices['Close'].rolling(window=30).mean()
dis_closing_prices['30dSTD'] = dis_closing_prices['Close'].rolling(window=30).std()

dis_closing_prices['Upper'] = dis_closing_prices['mavg30'] + (dis_closing_prices['30dSTD'] * 2)
dis_closing_prices['Lower'] = dis_closing_prices['mavg30'] - (dis_closing_prices['30dSTD'] * 2)

dis_closing_prices = dis_closing_prices.dropna()
dis_closing_prices.head()

In [ ]:
dis_closing_prices[['Close','mavg30','Upper','Lower']].plot(figsize=(20,10))
plt.grid(True)
plt.title('Disney Bollinger Bands (5 years)')
plt.axis('tight')
plt.ylabel('Price')
plt.show()